In [1]:
from jax import config
from dal_utils import print_result
config.update("jax_enable_x64", True)

In [2]:
import numpy as np
import numpy as np
from numpy.random import randn

rate, div, vol, spot, strike, barrier = 0.05, 0.03, 0.15, 100.0, 120.0, 150

def create_model():
    return np.array([rate, div, vol, spot, strike, barrier])

n_paths = 2 ** 20
T = 3.0

In [3]:
default_model = create_model()

## 0. DAL
------------------

In [4]:
# !pip install dal-python -U

from dal import *
import dal
dal.__version__

'2024.05.08'

In [5]:
today = Date_(2022, 9, 15)
EvaluationDate_Set(today)

maturity = today.AddDays(int(365 * T))
maturity
freq = "1M"

In [6]:
%%time
event_dates = ["STRIKE", "BARRIER", today]
events = [f"{strike:.2f}", f"{barrier:.2f}", "alive = 1"]
event_dates.append(f"START: {today} END: {maturity} FREQ: {freq}")
events.append("if spot() >= BARRIER:0.1 then alive = 0 end")
event_dates.append(maturity)
events.append(f"if spot() >= BARRIER:0.1 then alive = 0 end\ncall pays alive * MAX(spot() - STRIKE, 0.0)")

CPU times: total: 0 ns
Wall time: 0 ns


In [7]:
%%time

product = Product_New(event_dates, events)
model = BSModelData_New(spot, vol, rate, div)

# only price
res = MonteCarlo_Value(product, model, n_paths, "sobol", False, False)
dict(res)

CPU times: total: 1.34 s
Wall time: 81.6 ms


{'PV': 1.5057628229021947}

In [8]:
%%time

# price with derivatives
res = MonteCarlo_Value(product, model, n_paths, "sobol", False, True)
dict(res)

CPU times: total: 8.52 s
Wall time: 566 ms


{'PV': 1.5052428979007988,
 'd_BARRIER': 0.08925165480975476,
 'd_STRIKE': -0.1404746448629027,
 'd_div': -19.38122938204202,
 'd_rate': 14.865500688339626,
 'd_spot': 0.04974902216657963,
 'd_vol': -7.227015534880575}

## 1. DAL.JAX
----------------

In [9]:
import jax
import jax.numpy as jnp
import math

In [10]:
if freq == "1M":
    dt, NT = 1.0 / 12, 3 * 12
elif freq == "1W":
    dt, NT = 1.0 / 51, 3 * 51

In [11]:
@jax.jit
def update_state(Z, s, alive, not_alive, M, μ, d, σ, delta_t, log_b):
    s = s + (μ - d - 0.5 * σ * σ) * delta_t +  σ * jnp.sqrt(delta_t) * Z[0, :]
    alive = jnp.where(s >= log_b, not_alive, alive)
    return alive, s


def compute_barrier_price_jax(model, delta_t=dt, NT = NT, M=n_paths, key=jax.random.PRNGKey(1)):
    """
    Estimate the price of the up barrieroption using Monte Carlo.
    """
    # Set up
    μ, d, σ, S, K, b = model
    log_b = jnp.log(b)
    s = jnp.full(M, jnp.log(S))
    alive = jnp.full(M, 1.0)
    not_alive = jnp.full(M, 0.0)
    subkey = key
    for _ in range(NT):
        key, subkey = jax.random.split(subkey)
        Z = jax.random.normal(key, (1, M))
        alive, s = update_state(Z, s, alive, not_alive, M, μ, d, σ, delta_t, log_b)
    expectation = jnp.mean(alive * jnp.maximum(jnp.exp(s) - K, 0))
    return jnp.exp(-μ * delta_t * NT) * expectation

_ = compute_barrier_price_jax(default_model).block_until_ready()

In [12]:
%%time

# only price without jit
res = compute_barrier_price_jax(default_model).block_until_ready()
print_result(res)

CPU times: total: 172 ms
Wall time: 398 ms


{'PV': 1.5037587999606363}

In [13]:
compute_barrier_value_and_grad = jax.value_and_grad(compute_barrier_price_jax)
_ = compute_barrier_value_and_grad(default_model)[0].block_until_ready()

In [14]:
%%time

# price and derivatives without jit
# TODO: the output numbers of greeks are not correct
result = compute_barrier_value_and_grad(default_model, 1.0/ 12, 36, n_paths)
result[0].block_until_ready()
result[1].block_until_ready()
print_result(result, names=("rate", "div", "vol", "spot", "STRIKE", "barrier"))

CPU times: total: 109 ms
Wall time: 582 ms


{'PV': 1.5037587999606363,
 'd_rate': 50.37812987806875,
 'd_div': -54.889406277950656,
 'd_vol': 21.471054744003276,
 'd_spot': 0.18296468759316895,
 'd_STRIKE': -0.13993924966130214,
 'd_barrier': 0.0}